In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
from constants import GOV_API_KEY

In [ ]:
from flask_app.app import score

from notifications_python_client.notifications import NotificationsAPIClient

notifications_client = NotificationsAPIClient(GOV_API_KEY)

In [ ]:
import flask
from flask import Flask, request
import json
import pandas as pd
import pickle
import xgboost
from flask_app.extraction import url_extractor

import regex as re

import urllib.parse as urlparse
from tld import get_tld
import urllib.request as urlreq

In [ ]:
from functions.preprocessing import get_all_received_texts, get_spam_score, get_spam_classification

In [ ]:
from utils import NOT_SPAM_MSG_ID, SPAM_MSG_ID, SPAM_THRESHOLD

In [ ]:
google_url = "https://google.com"

google_spam_score = get_spam_score(google_url)

google_spam_score

In [ ]:
get_spam_classification(spam_score=google_spam_score, SPAM_THRESHOLD=SPAM_THRESHOLD)

In [ ]:
spam_url = "https://evri.id-60.com"

spam_eg_spam_score = get_spam_score(spam_url)

spam_eg_spam_score

In [ ]:
get_spam_classification(spam_score=spam_eg_spam_score, SPAM_THRESHOLD=SPAM_THRESHOLD)

In [ ]:
import time
texts = get_all_received_texts(notifications_client)

existing = []

for item in texts['received_text_messages']:
    existing.append(item['id'])
    print(item)

print(existing)

while True:

    # Get texts
    texts = get_all_received_texts(notifications_client)

    for item in texts['received_text_messages']:

        if item['id'] in existing:
            break

        print(item)
        
        try:
            # Get score and return text
            print("Getting: " + item['content'].lower())
            print(get_spam_score(item['content'].lower()))
            if get_spam_score(item['content'].lower()) > 0.75:
                
                # Return SPAM message
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=SPAM_MSG_ID, # required UUID string
                )

            else:

                # Return NOT SPAM
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=NOT_SPAM_MSG_ID, # required UUID string
                )

            #Add to existing
            existing.append(item['id'])
        except Exception as e:
            print("some sort of error occurred, let's hope it's not important")
            print(e)

    time.sleep(1)